In [1]:
import json
import numpy as np

In [2]:
import progressbar
from bert_serving.client import BertClient
bc = BertClient(ip='10.15.82.136')

In [3]:
root_folder = '/home/lyt/code/vqa-concept'
data_folder = '{}/dataTVQA'.format(root_folder)
fea_folder = '{}/image-feature/bottomup'.format(data_folder)
emb_size = 768
ocr_num = 50

In [4]:
with open('{}/popuchars.txt'.format(data_folder), 'r') as f:
    popuchars = f.read().splitlines()
print(popuchars)

['e', 'a', ' ', 'n', 'r', 'o', 'i', 's', 't', 'l', 'c', 'd', 'u', 'm', 'h', 'g', 'b', 'y', 'p', 'w', '1', '0', 'f', 'k', '2', 'v', '5', '3', '9', '4', 'x', '.', '8', '7', '6', 'j', ':', 'z', 'q', '-', ',', "'", '/', '&', '!', '$', '%', '"', '?', '+', '#', '(', ')', '@', ';', '*', '=', '_', '\\', '>', 'é']


In [5]:
#=======================================

In [6]:
split_name = 'train'
jsdata = json.load(open('{}/TextVQA_0.5_{}.json'.format(data_folder, split_name),'r'))
print(jsdata.keys())

dict_keys(['dataset_type', 'dataset_name', 'dataset_version', 'data'])


In [7]:
data = jsdata['data']
print(len(data),type(data))

34602 <class 'list'>


In [8]:
for pair in data:
    new_ocr_tokens = []
    for ocr in pair['ocr_tokens']:
        new_ocr = ''
        for c in ocr.lower():
            if c in popuchars:
                new_ocr += c
        if new_ocr.strip() != '':
            new_ocr_tokens.append(new_ocr)
    pair['ocr_tokens'] = new_ocr_tokens

In [9]:
json.dump(jsdata, open('{}/TextVQA_0.5_{}.clean.json'.format(data_folder, split_name),'w'))

In [10]:
#------save im_id:ocr_information------

In [11]:
im_ocr = {}
for pair in data:
    im_id = pair['image_id']
    if im_id not in im_ocr:
        im_ocr[im_id] = {'image_classes': pair['image_classes'],
                         'image_width': pair['image_width'],
                         'image_height': pair['image_height'],
                         'ocr_tokens': pair['ocr_tokens'],
                         'ocr_info': pair['ocr_info']}
#json.dump(im_ocr, open('{}/ocr_{}.json'.format(data_folder, split_name),'w'))

In [12]:
print(len(im_ocr))

21953


In [13]:
im_ids = np.load('{}/{}_100_id.npy'.format(fea_folder, split_name))
print(im_ids.shape)

(21953,)


In [14]:
#ocr_emb = np.random.randn(im_ids.shape[0], ocr_num, emb_size).astype('float32')
#ocr_emb = (ocr_emb + mean) * std
ocr_emb = np.zeros((im_ids.shape[0], ocr_num, emb_size), dtype='float32')
bar = progressbar.ProgressBar(max_value=im_ids.shape[0]).start()
for i,imid in enumerate(im_ids):
    bar.update(i+1)
    all_ocr = im_ocr[imid]['ocr_tokens']
    all_ocr = [ocr for ocr in all_ocr if ocr != '']
    num = min(ocr_num,len(all_ocr))
    if num == 0:
        continue
    ocr_list = [all_ocr[j].strip().lower() for j in range(num)]
    curr_embs = bc.encode(ocr_list)
    ocr_emb[i][:num] = curr_embs
bar.finish()

100% (21953 of 21953) |###################| Elapsed Time: 0:06:08 Time: 0:06:08


In [15]:
ocr_emb.shape

(21953, 50, 768)

In [16]:
np.save('{}/ocr_bert.new.{}/{}_{}_ocr.clean.0init.npy'.format(fea_folder, emb_size, split_name, ocr_num), ocr_emb)

In [17]:
#=======================================

In [18]:
bc.close()